## Import

In [100]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [101]:
# import torch

# # GPU 메모리 비우기
# torch.cuda.empty_cache()

# # 새로운 GPU 컨텍스트 생성
# with torch.cuda.device(0):
#     torch.cuda.empty_cache()


In [102]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [103]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [104]:
torch.cuda.is_available()

True

In [105]:
torch.__version__

'2.0.1+cu118'

In [106]:
# torch.cuda.empty_cache()

## Hyperparameter Setting

In [107]:
CFG = {
    'TRAIN_WINDOW_SIZE':120, # 90일치로 학습  초기는 90일이였음 
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':23,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':4096,
    'SEED':41
}

In [108]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### 데이터 불러오기

In [109]:
import pandas as pd
train_data = pd.read_csv('E:/LG/LG_data/train.csv').drop(columns=['ID', '제품',"소분류","중분류","대분류"])

In [110]:
train_data.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-00002,0,0,0,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,B002-00002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-00002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-00003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
import pandas as pd
brand_keyword_cnt = pd.read_csv('E:/LG/LG_data/brand_keyword_cnt.csv').drop(columns=[])

In [112]:
train_data.describe()

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
count,15890.000000,15890.000000,15890.00000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,...,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000,15890.000000
mean,12.887476,10.418880,9.01309,9.190938,11.204216,12.486281,12.933103,12.832599,13.326935,12.049843,...,0.501699,0.483260,0.473694,9.498175,9.370044,7.706293,7.123033,6.975456,9.040340,10.031026
std,183.612376,149.663362,95.82452,86.274138,92.072773,108.478567,135.615709,233.900622,242.761978,242.326264,...,8.853648,8.911203,8.567765,93.854552,90.632349,68.413621,68.629350,59.826757,73.637191,103.256072
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,15056.000000,14320.000000,6064.00000,4470.000000,6370.000000,8210.000000,9712.000000,24512.000000,24032.000000,24960.000000,...,700.000000,660.000000,660.000000,4308.000000,4596.000000,3960.000000,4224.000000,2700.000000,3045.000000,6048.000000


In [113]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15890 entries, 0 to 15889
Columns: 460 entries, 브랜드 to 2023-04-04
dtypes: int64(459), object(1)
memory usage: 55.8+ MB


In [114]:
brand_keyword_cnt.head(5)

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.84131,0.91383,1.450530,2.422390,1.871190,1.581080,1.232950,1.174930,1.145920,...,0.319110,0.391640,0.377130,0.49318,0.072520,0.29010,0.31911,0.232080,0.333620,0.44966
1,B002-00002,12.64868,20.27850,15.332170,12.750210,13.562510,13.707570,11.937910,15.564250,14.084710,...,10.269790,11.966920,10.646930,10.41485,10.487380,9.48651,9.28343,10.429350,11.154620,11.38671
2,B002-00003,0.33362,0.43516,0.362630,0.174060,0.217580,0.464170,0.420650,0.290100,0.377130,...,0.536690,0.696250,0.449660,0.39164,1.029880,0.49318,0.91383,0.797790,1.015370,0.88482
3,B002-00005,1.07339,1.71163,2.016240,1.914700,1.987230,2.146790,1.682620,1.378000,1.421520,...,2.219320,2.509420,2.872060,2.37888,2.030750,1.53756,1.34899,1.261960,2.320850,2.30635
4,B002-00006,0.00000,0.00000,0.188558,0.246574,0.246574,0.246574,0.377139,0.087012,0.261084,...,0.072526,0.290103,0.087012,0.00000,0.130542,0.00000,0.00000,0.072526,0.217577,0.00000


In [151]:
import pandas as pd

# Load the 'brand_keyword_cnt.csv' file
df_brand_keyword = pd.read_csv('E:/LG/LG_data/brand_keyword_cnt.csv')

# Fill NaN values with 0
df_brand_keyword = df_brand_keyword.fillna(0) 

# Convert float values (excluding the first column) to integers
for col in df_brand_keyword.columns[1:]:
    df_brand_keyword[col] = df_brand_keyword[col].astype(str).str.split('.').str[0].astype(int)

# Divide all values by 10
df_brand_keyword.iloc[:, 1:] = df_brand_keyword.iloc[:, 1:].div(5)

# Save the modified DataFrame back to the CSV file
df_brand_keyword.to_csv('brand_keyword_cnt_ints.csv', index=False)


In [152]:
brand_keyword_cnt_ints = pd.read_csv('E:/LG/brand_keyword_cnt_ints.csv')

In [153]:
brand_keyword_cnt_ints.head(5)

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.0,0.0,0.2,0.4,0.2,0.2,0.2,0.2,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B002-00002,2.4,4.0,3.0,2.4,2.6,2.6,2.2,3.0,2.8,...,2.0,2.2,2.0,2.0,2.0,1.8,1.8,2.0,2.2,2.2
2,B002-00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0
3,B002-00005,0.2,0.2,0.4,0.2,0.2,0.4,0.2,0.2,0.2,...,0.4,0.4,0.4,0.4,0.4,0.2,0.2,0.2,0.4,0.4
4,B002-00006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
# import pandas as pd
# import numpy as np

# # Load the two CSV files
# df_train = pd.read_csv('E:/LG/LG_data/train.csv').drop(columns=['ID', '제품' ]) # '소분류', '중분류', '대분류'
# brand_keyword_cnt_int = pd.read_csv('E:/LG/LG_data/brand_keyword_cnt.csv')

# # Create a dictionary to store the brand keyword values
# brand_keywords = df_brand_keyword.set_index('브랜드').to_dict('index')

# # Iterate through each row in 'df_train' and perform calculations
# result_data = []
# for idx, row in df_train.iterrows():
#     brand = row['브랜드']
#     if brand in brand_keywords:
#         keyword_values = brand_keywords[brand]
#         multiplied_values = [(row[col] * keyword_values.get(col, 1)) for col in df_train.columns[1:]]
#         result_data.append([brand] + multiplied_values)
#     else:
#         result_data.append(row.tolist())

# # Create a new DataFrame from the result data
# result_columns = df_train.columns.tolist()
# result_df = pd.DataFrame(result_data, columns=result_columns)

# # Save the result to a new CSV file
# result_df.to_csv('result.csv', index=False)


In [174]:
import pandas as pd
import numpy as np

# Load the two CSV files
df_train = pd.read_csv('E:/LG/LG_data/train.csv').drop(columns=['ID', '제품' ]) # '소분류', '중분류', '대분류'
brand_keyword_cnt_ints = pd.read_csv('E:/LG/brand_keyword_cnt_ints.csv')


# Create a dictionary to store the brand keyword values
brand_keywords = df_brand_keyword.set_index('브랜드').to_dict('index')



# Iterate through each row in 'df_train' and perform calculations
result_data = []
for idx, row in df_train.iterrows():
    brand = row['브랜드']
    if brand in brand_keywords:
        keyword_values = brand_keywords[brand]
        multiplied_values = []
        for col in df_train.columns[1:]:
            value = row[col]
            if isinstance(value, (int, float)):  # Check if the value is numeric
                multiplied_values.append(int(value) + keyword_values.get(col, 1))
            else:
                multiplied_values.append(value)  # Handle non-numeric values differently
        result_data.append([brand] + multiplied_values)
    else:
        result_data.append(row.tolist())

# Create a new DataFrame from the result data
result_columns = df_train.columns.tolist()
result_df = pd.DataFrame(result_data, columns=result_columns)

# Save the result to a new CSV file
result_df.to_csv('resultss.csv', index=False)


In [176]:
import pandas as pd
result_data = pd.read_csv('E:/LG/resultss.csv')
result_data.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,B002-C002-0007,B002-C003-0038,B002-00001,0.0,0.0,0.2,0.4,0.2,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B002-00002,B002-C002-0008,B002-C003-0044,B002-00002,2.4,4.0,3.0,2.4,2.6,2.6,...,2.0,2.2,2.0,3.0,5.0,3.8,1.8,2.0,4.2,2.2
2,B002-00002,B002-C002-0008,B002-C003-0044,B002-00002,2.4,4.0,3.0,2.4,2.6,2.6,...,2.0,2.2,2.0,2.0,2.0,1.8,1.8,2.0,2.2,2.2
3,B002-00002,B002-C002-0008,B002-C003-0044,B002-00002,2.4,4.0,3.0,2.4,2.6,2.6,...,2.0,2.2,2.0,2.0,2.0,1.8,1.8,2.0,2.2,2.2
4,B002-00003,B002-C002-0001,B002-C003-0003,B002-00003,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0


### 데이터 전처리

In [177]:
import pandas as pd

# 더 작은 값을 추가하여 분모가 0인 경우 방지하는 상수 정의
EPSILON = 1e-8

numeric_cols = result_data.columns[4:]


# 칵 column의 min 및 max 계산
min_values = result_data[numeric_cols].min(axis=1)
max_values = result_data[numeric_cols].max(axis=1)

# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 EPSILON 더해주기
ranges = max_values - min_values + EPSILON

# min-max scaling 수행
scaled_data = (result_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)

# 스케일링된 데이터로 업데이트
result_data[numeric_cols] = scaled_data

# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()



print(result_data.head())



          대분류             중분류             소분류         브랜드  2022-01-01  \
0  B002-00001  B002-C002-0007  B002-C003-0038  B002-00001    0.000000   
1  B002-00002  B002-C002-0008  B002-C003-0044  B002-00002    0.142857   
2  B002-00002  B002-C002-0008  B002-C003-0044  B002-00002    0.035176   
3  B002-00002  B002-C002-0008  B002-C003-0044  B002-00002    0.017544   
4  B002-00003  B002-C002-0001  B002-C003-0003  B002-00003    0.000000   

   2022-01-02  2022-01-03  2022-01-04  2022-01-05  2022-01-06  ...  \
0    0.000000    0.016393    0.032787    0.016393    0.016393  ...   
1    0.306122    0.204082    0.142857    0.163265    0.163265  ...   
2    0.075377    0.050251    0.035176    0.040201    0.040201  ...   
3    0.037594    0.025063    0.017544    0.020050    0.020050  ...   
4    0.000000    0.000000    0.000000    0.000000    0.000000  ...   

   2023-03-26  2023-03-27  2023-03-28  2023-03-29  2023-03-30  2023-03-31  \
0    0.000000    0.000000    0.000000    0.000000    0.000000  

In [178]:
result_data.head(5)

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,B002-C002-0007,B002-C003-0038,B002-00001,0.000000,0.000000,0.016393,0.032787,0.016393,0.016393,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,B002-00002,B002-C002-0008,B002-C003-0044,B002-00002,0.142857,0.306122,0.204082,0.142857,0.163265,0.163265,...,0.102041,0.122449,0.102041,0.204082,0.408163,0.285714,0.081633,0.102041,0.326531,0.122449
2,B002-00002,B002-C002-0008,B002-C003-0044,B002-00002,0.035176,0.075377,0.050251,0.035176,0.040201,0.040201,...,0.025126,0.030151,0.025126,0.025126,0.025126,0.020101,0.020101,0.025126,0.030151,0.030151
3,B002-00002,B002-C002-0008,B002-C003-0044,B002-00002,0.017544,0.037594,0.025063,0.017544,0.020050,0.020050,...,0.012531,0.015038,0.012531,0.012531,0.012531,0.010025,0.010025,0.012531,0.015038,0.015038
4,B002-00003,B002-C002-0001,B002-C003-0003,B002-00003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.004348,0.000000,0.000000,0.000000,0.004348,0.000000


In [179]:
# # Data Scaling
# scale_max_dict = {}
# scale_min_dict = {}

# for idx in tqdm(range(len(train_data))):
#     maxi = np.max(train_data.iloc[idx,4:])
#     mini = np.min(train_data.iloc[idx,4:])
    
#     if maxi == mini :
#         train_data.iloc[idx,4:] = 0
#     else:
#         train_data.iloc[idx,4:] = (train_data.iloc[idx,4:] - mini) / (maxi - mini)
    
#     scale_max_dict[idx] = maxi
#     scale_min_dict[idx] = mini

In [180]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류','브랜드'] # '대분류', '중분류', '소분류',

for col in categorical_columns:
    label_encoder.fit(result_data[col])
    result_data[col] = label_encoder.transform(result_data[col])

In [181]:
def make_result_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [182]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [183]:
result_data.tail()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
15885,3169,7,41,3169,0.001280,0.002561,0.003841,0.003841,0.005122,0.003841,...,0.002561,0.003841,0.003841,0.002561,0.001280,0.002561,0.00000,0.001280,0.002561,0.002561
15886,3169,7,43,3169,0.001626,0.003252,0.004878,0.004878,0.006504,0.004878,...,0.003252,0.004878,0.004878,0.027642,0.001626,0.019512,0.03252,0.009756,0.011382,0.027642
15887,3169,7,43,3169,0.007634,0.015267,0.022901,0.022901,0.030534,0.022901,...,0.015267,0.022901,0.022901,0.015267,0.007634,0.015267,0.00000,0.007634,0.015267,0.015267
15888,3169,7,43,3169,0.013889,0.027778,0.041667,0.041667,0.055556,0.041667,...,0.027778,0.041667,0.041667,0.027778,0.013889,0.027778,0.00000,0.013889,0.027778,0.166667
15889,3169,3,19,3169,0.125000,0.250000,0.375000,0.375000,0.500000,0.375000,...,0.250000,0.375000,0.375000,0.250000,0.125000,0.250000,0.00000,0.125000,0.250000,0.250000


In [184]:
train_input, train_target = make_result_data(result_data)
test_input = make_predict_data(result_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

100%|██████████| 15890/15890 [00:24<00:00, 660.49it/s]


In [185]:
data_len = len(train_input)
val_ratio = 0.1
test_ratio = 0.1

val_len = int(data_len * val_ratio)
test_len = int(data_len * test_ratio)

val_input = train_input[-val_len:]
val_target = train_target[-val_len:]


train_input = train_input[:-val_len - test_len]
train_target = train_target[:-val_len - test_len]


In [186]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4105976, 120, 5),
 (4105976, 21),
 (513247, 120, 5),
 (513247, 21),
 (15890, 120, 5))

### Custom Dataset

In [187]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [188]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 모델 선언

## 개선된 gru

In [189]:
class ImprovedModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE'], dropout_prob=0.7):
        super(ImprovedModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.ln = nn.LayerNorm(hidden_size)  # Layer Normalization
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, output_size)
        )
        self.actv = nn.ReLU()

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        gru_out, hidden = self.gru(x, hidden)
        gru_out = self.dropout(gru_out)
        gru_out = self.ln(gru_out)  # Applying Layer Normalization

        last_output = gru_out[:, -1, :]

        output = self.actv(self.fc(last_output))

        return output.squeeze(1)
    def init_hidden(self, batch_size, device):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)



## lstm으로 만든것

In [190]:
# import torch
# import torch.nn as nn

# class ImprovedModel(nn.Module):
#     def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE'], dropout_prob=0.2):
#         super(ImprovedModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)  # Change nn.GRU to nn.LSTM
#         self.ln = nn.LayerNorm(hidden_size)  # Layer Normalization
#         self.dropout = nn.Dropout(dropout_prob)
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size, hidden_size // 2),
#             nn.ReLU(),
#             nn.Linear(hidden_size // 2, output_size)
#         )
#         self.actv = nn.ReLU()

#     def forward(self, x):
#         batch_size = x.size(0)
#         hidden = self.init_hidden(batch_size, x.device)  # Use x.device instead of 'device'

#         lstm_out, (hidden, cell) = self.lstm(x, (hidden, hidden))  # Use the same hidden tensor for both hidden and cell states
#         lstm_out = self.dropout(lstm_out)
#         lstm_out = self.ln(lstm_out)  # Applying Layer Normalization

#         last_output = lstm_out[:, -1, :]

#         output = self.actv(self.fc(last_output))

#         return output.squeeze(1)
    
#     def init_hidden(self, batch_size, device):
#         return (torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device),
#                 torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device))  # Initializing hidden and cell states for LSTM

### 모델 학습

In [191]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    
    
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        

        # # 학습 루프 안에서
        # if best_loss > val_loss:
        #     best_loss = val_loss
        #     best_model = model
        #     print('Model Saved')

        # 학습이 끝난 후
    return model  # 모든 모델을 반환


In [192]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
            
    return np.mean(val_loss)

## Run !!

## 과적합 일어나서 드롭아웃 0.7로 올림

In [193]:
model = ImprovedModel() # BaseModel() 
optimizer = torch.optim.RAdam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

100%|██████████| 126/126 [00:32<00:00,  3.92it/s]


Epoch : [1] Train Loss : [0.03174] Val Loss : [0.02976]


100%|██████████| 126/126 [00:32<00:00,  3.89it/s]


Epoch : [2] Train Loss : [0.03021] Val Loss : [0.03025]


100%|██████████| 126/126 [00:31<00:00,  4.04it/s]


...

Epoch : [6] Train Loss : [0.02965] Val Loss : [0.04208]


100%|██████████| 126/126 [00:29<00:00,  4.29it/s]


Epoch : [7] Train Loss : [0.02994] Val Loss : [0.03035]


 85%|████████▌ | 856/1003 [07:15<01:14,  1.97it/s]


KeyboardInterrupt: 

## 모델 추론

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [ ]:
# pred = inference(infer_model, test_loader, device)
pred = inference(infer_model, test_loader, device)

100%|██████████| 4/4 [00:00<00:00,  5.76it/s]


In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :]  * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

(15890, 21)

## Submission

In [ ]:
submit = pd.read_csv('E:/LG/LG_data/sample_submission.csv')
submit.tail()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
15885,15885,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15886,15886,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15887,15887,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15888,15888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15889,15889,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
submit.iloc[:,1:] = pred
submit.tail()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
15885,15885,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
15886,15886,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
15887,15887,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
15888,15888,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
15889,15889,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
submit.to_csv('E:/LG/LG_data/models/baseline_submit_epoch_1_90_4096_512_RAdam_GRU_02_LAYER_29999_.csv', index=False)